In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import skorch
import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

import numpy as np
from Lung_dataset import ILDDataset

In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


In [3]:
# #load the data
# x = np.load("./large_data/X.npy")
# y = np.load("./large_data/y.npy")

# x_full = torch.from_numpy(x)
# y_full = torch.from_numpy(y)
# x_full = x_full.to(device=device, dtype=dtype)
# y_full = y_full.to(device=device, dtype=torch.long)

In [4]:
NUM_TOTAL = 1982
NUM_TRAIN = 1200
#add path as absolute path for root dir
lung_dataset_train = ILDDataset(csv_file='../../Data/Clean_train_test_dataset/Dataset/train_labels.csv', 
                          root_dir='/Users/magdy/Desktop/BMI260/Project/Data/Clean_train_test_dataset/Dataset/train', train=True)#, transform=transform)


#add path as absolute path for root dir
lung_dataset_test = ILDDataset(csv_file='../../Data/Clean_train_test_dataset/Dataset/test_labels.csv', 
                          root_dir='/Users/magdy/Desktop/BMI260/Project/Data/Clean_train_test_dataset/Dataset/test/', train=False)#, transform=transform)


loader_train = DataLoader(lung_dataset_train, batch_size=32, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

loader_val = DataLoader(lung_dataset_train, batch_size=32, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, NUM_TOTAL)))

loader_test = DataLoader(lung_dataset_test, batch_size=32)

In [5]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)


In [6]:
channel_1 = 64
channel_2 = 64
channel_3 = 128

learning_rate = 2.5e-3

in_channel = 62
num_classes = 3

# model = nn.Sequential(
#     nn.Conv2d(in_channel, channel_1, 5, padding=2),
#     nn.BatchNorm2d(channel_1),
#     nn.ReLU(),
#     nn.Conv2d(channel_1, channel_2, 3, padding=1),
#     nn.BatchNorm2d(channel_2),
#     nn.ReLU(),
#     nn.MaxPool2d(2),
#     Flatten(),
#     nn.Linear((32*512*512)/2, num_classes)
# )
# model = model.to(device=device)

model = models.resnet50(pretrained=True)
model.avgpool = nn.AvgPool2d(1, stride=1)
for param in model.parameters():
    param.requires_grad = False
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=1, bias=False)
model.fc = nn.Linear(8192, num_classes)

# from skorch import NeuralNetClassifier

# net = NeuralNetClassifier(
#     module=model,
#     criterion = nn.CrossEntropyLoss,
#     optimizer=optim.Adam,
#     train_split=None,
#     max_epochs=5,
#     lr = learning_rate,
#     warm_start = True,
#     device = device
# )

In [7]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            x.unsqueeze_(1)
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [8]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            
            x.unsqueeze_(1)
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [9]:
# optimizer = optim.Adam(model.parameters(), lr = 0.0001)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)
train_part34(model, optimizer, epochs=10)

Iteration 0, loss = 1.1867
Checking accuracy on validation set
Got 198 / 782 correct (25.32)

Iteration 0, loss = 0.9206
Checking accuracy on validation set
Got 198 / 782 correct (25.32)

Iteration 0, loss = 0.8316
Checking accuracy on validation set
Got 190 / 782 correct (24.30)

Iteration 0, loss = 2.6572
Checking accuracy on validation set
Got 198 / 782 correct (25.32)

Iteration 0, loss = 0.9563
Checking accuracy on validation set
Got 199 / 782 correct (25.45)



KeyboardInterrupt: 

In [ ]:
# from sklearn.model_selection import KFold
# from sklearn import metrics

# # for epoch in range(5):
# kf = KFold(n_splits=3, shuffle = True)
# accuracies=[]
# for train_index, test_index in kf.split(x):
#     accuracies=[]
# #     (N, C1, C2, S) = x.shape 
# #     x = x.reshape((N, 62, 512, 512))
# #     y = y.reshape((y.shape[0]))
# #     print(x.shape)
# #     print(y.shape)
# #     print(train_index)
# #     print(test_index)
#     xk_train, xk_test = x[train_index], x[test_index]
#     yk_train, yk_test = y[train_index], y[test_index]
#     net.fit(xk_train,yk_train)
#     y_pred = net.predict(xk_test)
#     acc = metrics.accuracy_score(yk_test, y_pred)
#     accuracies.append(acc)
#     print('FinalAccuracy %.4f' % (np.mean(accuracies)))

In [ ]:
(N, C1, C2, S) = x_Test.shape
x_Test = x_Test.reshape((N, S, C1, C2))
y_pred_test = net.predict(x_Test)
acc = metrics.accuracy_score(y_Test, y_pred_test)
print('TestAccuracy %.4f' % (acc))